# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


## Importing encoded and clean database

In [3]:
import pandas as pd

# Carga el archivo con configuraciones ajustadas porque no tengo espaciooo en mi computadora
df_enclean = pd.read_csv(
    r"C:\Users\Usuario\OneDrive - Instituto Tecnologico y de Estudios Superiores de Monterrey\Documents\TEC\6to semestre\optimizacion MA2008B\reto ma2008b\data\df_encoded_2.0_FechasIguales.csv",
    delimiter=",",
    encoding="latin-1", 
    low_memory=True,
    engine="python"
)


In [4]:
# Cargar datos desde un archivo csv a un dataframe
#df_enclean = pd.read_csv("/Users/albaruiz/Downloads/Sexto semestre/Análisis numérico para la optimización no-lineal/data/df_encoded_2.0_FechasIguales.csv", delimiter=",", encoding="latin-1", low_memory=False)
# Hacer dataframe
df_enclean = pd.DataFrame(df_enclean)
df_enclean.head()

,ORG,Fecha_aprobacion,Fecha_activacion,Mob,Saldo_total,Saldo_Mes,Pago_minimo,Utilizacion,Corte,Fecha_corte,...,ESTADO_SIN,ESTADO_SLP,ESTADO_SON,ESTADO_TAB,ESTADO_TAM,ESTADO_TLA,ESTADO_TLX,ESTADO_VER,ESTADO_YUC,ESTADO_ZAC
0,310,2494,2537,199,1533.90,734.40,115.0,0.174307,20,8540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,310,2725,2726,191,913.02,747.02,194.0,0.332007,20,8540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,310,2751,2751,190,5703.77,3942.12,553.0,0.769740,15,8535,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,310,2621,2622,194,2653.88,2653.88,298.0,0.288465,10,8530,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,310,2653,2654,193,3466.46,1328.87,313.0,0.376789,3,8523,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Wrapper: Feature Selection with RFECV

In [5]:
# importar csv encoded
df_encoded = df_enclean.copy()
# Hacer dataframe
df_encoded = pd.DataFrame(df_encoded)
df_encoded.head()

,ORG,Fecha_aprobacion,Fecha_activacion,Mob,Saldo_total,Saldo_Mes,Pago_minimo,Utilizacion,Corte,Fecha_corte,...,ESTADO_SIN,ESTADO_SLP,ESTADO_SON,ESTADO_TAB,ESTADO_TAM,ESTADO_TLA,ESTADO_TLX,ESTADO_VER,ESTADO_YUC,ESTADO_ZAC
0,310,2494,2537,199,1533.90,734.40,115.0,0.174307,20,8540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,310,2725,2726,191,913.02,747.02,194.0,0.332007,20,8540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,310,2751,2751,190,5703.77,3942.12,553.0,0.769740,15,8535,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,310,2621,2622,194,2653.88,2653.88,298.0,0.288465,10,8530,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,310,2653,2654,193,3466.46,1328.87,313.0,0.376789,3,8523,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
x = df_encoded.drop(columns=['Variable_objetivo']).values
y = df_encoded['Variable_objetivo'].values
features = df_encoded.drop(columns=['Variable_objetivo']).columns
n_features = len(features)
print(f"Características:\n {features}")
print(f"Total de características: {n_features}")

Características:
 Index(['ORG', 'Fecha_aprobacion', 'Fecha_activacion', 'Mob', 'Saldo_total',
       'Saldo_Mes', 'Pago_minimo', 'Utilizacion', 'Corte', 'Fecha_corte',
       ...
       'ESTADO_SIN', 'ESTADO_SLP', 'ESTADO_SON', 'ESTADO_TAB', 'ESTADO_TAM',
       'ESTADO_TLA', 'ESTADO_TLX', 'ESTADO_VER', 'ESTADO_YUC', 'ESTADO_ZAC'],
      dtype='object', length=144)
Total de características: 144


In [ ]:
print("----- Optimal selection of number of features with RFECV -----")

# Configuración de RandomForest y RFECV
clf = RandomForestClassifier(n_estimators=100, random_state=1234)
kf = StratifiedKFold(n_splits=5, shuffle=True)

# RFECV para seleccionar los mejores features
fselection_cv = RFECV(estimator=clf, step=10, cv=kf, scoring='accuracy', n_jobs=-1, verbose=1)
fselection_cv.fit(x, y)

----- Optimal selection of number of features with RFECV -----


In [ ]:
# Resultados de RFECV
print("Optimal number of features: ", fselection_cv.n_features_)
selected_features = df_encoded.drop(columns=['Variable_objetivo']).columns[fselection_cv.support_]
print("Best features (support mask): ", selected_features.tolist())

# Transformar los datos con los mejores features
x_transformed = fselection_cv.transform(x)

# Validación cruzada para evaluar rendimiento con los mejores features
acc_cv = []
for train_index, test_index in kf.split(x_transformed, y):
    x_train, x_test = x_transformed[train_index], x_transformed[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)

    acc_cv.append(accuracy_score(y_test, y_pred))

# Precisión promedio
acc = np.average(acc_cv)
print("Average accuracy with selected features: ", acc)

Optimal number of features:  104
Best features (support mask):  ['ORG', 'Fecha_aprobacion', 'Fecha_activacion', 'Mob', 'Saldo_total', 'Saldo_Mes', 'Pago_minimo', 'Utilizacion', 'Corte', 'Fecha_corte', 'Fecha_limite_pago', 'Fecha_prox_corte', 'Ciclo_Atraso', 'Limite_credito', 'Fecha_nac', 'CP', 'Saldo_total_M1', 'Saldo_total_M2', 'Saldo_total_M3', 'Saldo_total_M4', 'Saldo_total_M5', 'Saldo_total_M6', 'Saldo_Mes_M1', 'Saldo_Mes_M2', 'Saldo_Mes_M3', 'Saldo_Mes_M4', 'Saldo_Mes_M5', 'Saldo_Mes_M6', 'Pago_minimo_M1', 'Pago_minimo_M2', 'Pago_minimo_M3', 'Pago_minimo_M4', 'Pago_minimo_M5', 'Pago_minimo_M6', 'Fecha_corte_M1', 'Fecha_corte_M2', 'Fecha_corte_M3', 'Fecha_corte_M4', 'Fecha_corte_M5', 'Fecha_corte_M6', 'Fecha_limite_pago_M1', 'Fecha_limite_pago_M2', 'Fecha_limite_pago_M3', 'Fecha_limite_pago_M4', 'Fecha_limite_pago_M5', 'Fecha_limite_pago_M6', 'Fecha_prox_corte_M1', 'Ciclo_atraso_M1', 'Ciclo_atraso_M2', 'Ciclo_atraso_M3', 'Ciclo_atraso_M4', 'Ciclo_atraso_M5', 'Ciclo_atraso_M6', 'Uti

## Principal Component Analysis (PCA)

In [ ]:
data = df_enclean

# Separar características y variable objetivo
X = data.drop(columns=['Variable_objetivo'])
y = data['Variable_objetivo']

# Seleccionar solo columnas numéricas
continuous_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
X = X[continuous_cols]

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Estandarizar los datos
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# Determinar el número óptimo de componentes con PCA (90% de varianza)
pca_full = PCA().fit(X_train_std)
explained_variance = np.cumsum(pca_full.explained_variance_ratio_)

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(explained_variance)+1), explained_variance, marker="o", linestyle="--")
plt.axhline(y=0.90, color="g", linestyle="--", label="90% varianza")
plt.xlabel("Número de Componentes")
plt.ylabel("Varianza Explicada Acumulada")
plt.title("Selección del Número Óptimo de Componentes (Entrenamiento)")
plt.legend()
plt.show()

optimal_components = np.argmax(explained_variance >= 0.90) + 1
print(f"Número óptimo de componentes para capturar 90% de varianza: {optimal_components}")

# Aplicar PCA con el número óptimo de componentes
pca = PCA(n_components=optimal_components)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

# Varianza explicada por cada componente
explained_variance_ratio = pca.explained_variance_ratio_
print(f"Varianza explicada por componente: {explained_variance_ratio}")

# Método del codo para determinar el número óptimo de clusters
wcss = []
K_range = range(1, 15)
for k in K_range:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(X_train_pca)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(K_range, wcss, 'o--', markersize=6)
plt.xlabel("Número de Clusters")
plt.ylabel("WCSS (Inercia)")
plt.title("Método del Codo para Determinar K (Entrenamiento)")
plt.show()

# Seleccionar el número óptimo de clusters ( según la gráfica) 
n_clusters = 5  #puede ser entre 4-6

# Clustering con K-Means
kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
labels_train = kmeans.fit_predict(X_train_pca)

# Visualización en 2D usando los dos primeros componentes
pca_2d = PCA(n_components=2)
X_train_pca_2d = pca_2d.fit_transform(X_train_std)

pca_df = pd.DataFrame(X_train_pca_2d, columns=["PC1", "PC2"])
pca_df["Cluster"] = labels_train

plt.figure(figsize=(8, 6))
sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue="Cluster", palette="tab10", alpha=0.6, edgecolor=None, s=20)
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.title("Segmentación con PCA y K-Means (Entrenamiento)")
plt.legend(title="Cluster")
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

# Crear y exportar DataFrame con PCA y clusters
df_pca_train = pd.DataFrame(X_train_pca, columns=[f'PCA_{i+1}' for i in range(optimal_components)])
df_pca_train['Cluster'] = labels_train
df_pca_train['Variable_objetivo'] = y_train.reset_index(drop=True)
df_pca_train.to_csv("df_pca_train_with_clusters.csv", index=False)

print("El DataFrame de entrenamiento con PCA y clusters ha sido exportado exitosamente.")

# Model Training

## Random Forest

In [ ]:
############# Entrenamiento de modelo RF #############
clf = RandomForestClassifier(n_estimators=100, random_state=1234)
clf.fit(X_train_pca, y_train)

# Predecir en los datos de prueba
y_pred = clf.predict(X_test_pca)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (Random Forest) with PCA: {accuracy}")

Accuracy (Random Forest) with PCA: 0.7478330981656924


## Support Vector Machine (SVM)

In [ ]:
############# Entrenamiento de modelo SVM #############
svm = SVC(kernel='rbf', C=1, gamma='scale', random_state=1234)
svm.fit(X_train_pca, y_train)

# Predecir
y_pred = svm.predict(X_test_pca)

# Calcular precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (SVM) with PCA: {accuracy}")

# Plotting the explained variance
plt.figure(figsize=(10, 6))
plt.plot(np.cumsum(explained_variance))
plt.xlabel('Number of components')
plt.ylabel('Explained variance')
plt.title('Explained variance by number of components')
plt.grid()
plt.show()

# Fine-tuning of the models

## Random Forest

In [ ]:
clf = RandomForestClassifier(random_state=1234)

# Espacio de hiperparámetros
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [None, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=20,          # Solo 20 combinaciones aleatorias
    scoring='accuracy',
    cv=3,               # 3 folds de validación cruzada
    verbose=2,
    n_jobs=-1,
    random_state=1234
)

# Entrenar
random_search.fit(X_train_pca, y_train)

# Obtener el mejor modelo
best_model = random_search.best_estimator_

# Predecir
y_pred = best_model.predict(X_test_pca)

# Evaluar
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (Random Forest after fine-tuning): {accuracy}")
print(f"Mejores hiperparámetros: {random_search.best_params_}")


## Support Vector Machine (SVM)